In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
import json
import ast
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
%matplotlib inline

In [5]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
q = """
select * from datasets.hex_adjacency
"""

df_adjacency_copy = pd.read_sql(q, connection)
df_adjacency_copy.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
0,88609c3693fffff-Monday-morning_peak,Monday,morning_peak,88609c3693fffff,"['88609c369bfffff', '88609dcb47fffff', '88609d...","['88609c369bfffff', '88609dcb47fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
1,88609dd9b1fffff-Monday-rest_evening,Monday,rest_evening,88609dd9b1fffff,"['88609dd9bbfffff', '88609dd9b9fffff', '88609d...","['88609dd9bbfffff', '88609dd9b9fffff', '88609d...",adding_new_cities,22:00,23:59,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
2,88609dd9b5fffff-Monday-morning_peak,Monday,morning_peak,88609dd9b5fffff,"['88609dd9b9fffff', '88609dcb43fffff', '88609d...","['88609dd9b9fffff', '88609dcb43fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
3,88609c36edfffff-Wednesday-afternoon,Wednesday,afternoon,88609c36edfffff,"['88609c36e5fffff', '88609c3615fffff', '88609c...","['88609c36e5fffff', '88609c3615fffff', '88609c...",adding_new_cities,12:00,17:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
4,88609c3697fffff-Tuesday-rest_morning,Tuesday,rest_morning,88609c3697fffff,"['88609c369bfffff', '88609c36bbfffff', '88609c...","['88609c369bfffff', '88609c36bbfffff', '88609c...",adding_new_cities,00:00,08:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad


In [4]:
df_adjacency_copy.shape

(7353465, 24)

In [5]:
df_adjacency_copy.dtypes

uid                                              object
dayname                                          object
time_period                                      object
source_hex                                       object
adjacency                                        object
backup_adjacency                                 object
type                                             object
start_ts                                         object
end_ts                                           object
city_id                                          object
date                                             object
propagation_type                                 object
propagation_eta_buckets                          object
multicast_max_batch_size                         object
service_detail_id                                object
undesirable_hexes                                object
max_mmo_parallel_order                            int64
propagation_score_buckets                       

In [6]:
df_adjacency = df_adjacency_copy.copy()
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
0,88609c3693fffff-Monday-morning_peak,Monday,morning_peak,88609c3693fffff,"['88609c369bfffff', '88609dcb47fffff', '88609d...","['88609c369bfffff', '88609dcb47fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
1,88609dd9b1fffff-Monday-rest_evening,Monday,rest_evening,88609dd9b1fffff,"['88609dd9bbfffff', '88609dd9b9fffff', '88609d...","['88609dd9bbfffff', '88609dd9b9fffff', '88609d...",adding_new_cities,22:00,23:59,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
2,88609dd9b5fffff-Monday-morning_peak,Monday,morning_peak,88609dd9b5fffff,"['88609dd9b9fffff', '88609dcb43fffff', '88609d...","['88609dd9b9fffff', '88609dcb43fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
3,88609c36edfffff-Wednesday-afternoon,Wednesday,afternoon,88609c36edfffff,"['88609c36e5fffff', '88609c3615fffff', '88609c...","['88609c36e5fffff', '88609c3615fffff', '88609c...",adding_new_cities,12:00,17:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
4,88609c3697fffff-Tuesday-rest_morning,Tuesday,rest_morning,88609c3697fffff,"['88609c369bfffff', '88609c36bbfffff', '88609c...","['88609c369bfffff', '88609c36bbfffff', '88609c...",adding_new_cities,00:00,08:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad


In [12]:
sourcing_types = df_adjacency['type'].unique().tolist()
for type in sourcing_types:
    if type.startswith('custom'):
        print(type)

custom_sourcing_bangalore_2023_hex_removal_auto
custom_sourcing_bangalore_auto_hsr_lower_2023
custom_sourcing_bangalore_auto_hsr_upper_right_2023
custom_sourcing_bangalore_2023_hex_removal
custom_sourcing_bangalore_2023


In [21]:
df_adjacency[(df_adjacency['source_hex'] == '883da11ab5fffff') & (df_adjacency['time_period'] == 'morning_peak') & 
             (df_adjacency['service_detail_id'] == '5da4660028af187d8d52cc3c')]['adjacency'].values[0]

"['883da11ab1fffff', '883da11ab7fffff', '883da118cbfffff', '883da11abdfffff', '883da11aa3fffff', '883da11ab5fffff', '883da118c9fffff']"

In [24]:
df_adjacency[df_adjacency['type'] == 'bad_hex_2.0']['city_name'].unique()

array(['kolkata', 'jaipur', 'bangalore', 'hyderabad', 'chennai',
       'bhubaneswar', 'mumbai', 'pune', 'coimbatore', 'delhi', 'indore',
       'lucknow', 'vijayawada', 'ahmedabad', 'vishakapatnam', 'patna',
       'ludhiana', 'bhopal', 'guwahati'], dtype=object)

In [22]:
h3.k_ring('883da11ab5fffff', 1)

{'883da118c9fffff',
 '883da118cbfffff',
 '883da11aa3fffff',
 '883da11ab1fffff',
 '883da11ab5fffff',
 '883da11ab7fffff',
 '883da11abdfffff'}

In [13]:
df_adjacency['adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023_hex_removal_auto',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 1)))), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023_hex_removal_auto',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 2)))), df_adjacency['backup_adjacency'])

In [14]:
df_adjacency['adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_lower_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 1)))), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_lower_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 2)))), df_adjacency['backup_adjacency'])

In [15]:
df_adjacency['adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_upper_right_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 1)))), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_upper_right_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 2)))), df_adjacency['backup_adjacency'])

In [16]:
df_adjacency['adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023_hex_removal',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 1)))), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023_hex_removal',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 2)))), df_adjacency['backup_adjacency'])

In [17]:
df_adjacency['adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 1)))), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['type'] == 'custom_sourcing_bangalore_2023',
                                       df_adjacency['source_hex'].apply(lambda x: str(list(h3.k_ring(x, 2)))), df_adjacency['backup_adjacency'])

In [18]:
df_adjacency['type'] = np.where(df_adjacency['type'].isin(['custom_sourcing_bangalore_2023_hex_removal_auto', 'custom_sourcing_bangalore_auto_hsr_lower_2023',
                                                           'custom_sourcing_bangalore_auto_hsr_upper_right_2023', 'custom_sourcing_bangalore_2023_hex_removal',
                                                           'custom_sourcing_bangalore_2023']), 'reset_to_original', df_adjacency['type'])

In [19]:
df_adjacency.to_parquet('pan_india_updated_adjacency_20230522.parquet', index = False)

In [19]:
df_adjacency = pd.read_parquet('pan_india_updated_adjacency_20230522.parquet')
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
0,88609c3693fffff-Monday-morning_peak,Monday,morning_peak,88609c3693fffff,"['88609c369bfffff', '88609dcb47fffff', '88609d...","['88609c369bfffff', '88609dcb47fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
1,88609dd9b1fffff-Monday-rest_evening,Monday,rest_evening,88609dd9b1fffff,"['88609dd9bbfffff', '88609dd9b9fffff', '88609d...","['88609dd9bbfffff', '88609dd9b9fffff', '88609d...",adding_new_cities,22:00,23:59,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
2,88609dd9b5fffff-Monday-morning_peak,Monday,morning_peak,88609dd9b5fffff,"['88609dd9b9fffff', '88609dcb43fffff', '88609d...","['88609dd9b9fffff', '88609dcb43fffff', '88609d...",adding_new_cities,08:00,12:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
3,88609c36edfffff-Wednesday-afternoon,Wednesday,afternoon,88609c36edfffff,"['88609c36e5fffff', '88609c3615fffff', '88609c...","['88609c36e5fffff', '88609c3615fffff', '88609c...",adding_new_cities,12:00,17:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad
4,88609c3697fffff-Tuesday-rest_morning,Tuesday,rest_morning,88609c3697fffff,"['88609c369bfffff', '88609c36bbfffff', '88609c...","['88609c369bfffff', '88609c36bbfffff', '88609c...",adding_new_cities,00:00,08:00,5b4e0c8f5f24742f74ee51f1,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5b4e0c6edcbfc67980999f05,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,aurangabad


In [7]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Delhi')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [9]:
df_cluster_copy[df_cluster_copy['cluster'] == 'M Iffco Metro']

,city,cluster,hex_id,cluster_original,resolution,executiondate
700,Delhi,M Iffco Metro,883da11a8bfffff,M Iffco Metro,8,2023-05-23
3770,Delhi,M Iffco Metro,883da11ad7fffff,M Iffco Metro,8,2023-05-23


In [3]:
def plot_hexes(list_of_hexes):
    from keplergl import KeplerGl  
    map_1 = KeplerGl(height=500)   
    for i in range(0,len(list_of_hexes)): 
        map_1.add_data( data = pd.DataFrame({'hex_id':[list_of_hexes[i]]}), name = 'data_'+str(i))  
    return map_1

In [32]:
%matplotlib inline

In [5]:
plot_hexes(['883da11a8bfffff', '883da11ad5fffff', '883da11a9dfffff', '883da11a89fffff', '883da11a83fffff', '883da11ad7fffff', '883da11a81fffff'])

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_0': {'index': [0], 'columns': ['hex_id'], 'data': [['883da11a8bfffff']]}, 'data_1': {'ind…

In [13]:
df_adjacency[(df_adjacency['source_hex'] == '883da11a8bfffff') & (df_adjacency['service_detail_id'] == '5fc8c40bf4f0a70007d7cfbc')]['adjacency'].values[0]

"['883da11a8bfffff', '883da11ad5fffff', '883da11a9dfffff', '883da11a89fffff', '883da11a83fffff', '883da11ad7fffff', '883da11a81fffff']"

In [13]:
#Load the hexes for hyderabad morning zone

hexes_adjacency_to_change_lower = pd.read_csv('lower_half_inner.csv')
hexes_adjacency_to_change_upper = pd.read_csv('upper_right_inner.csv')
hexes_adjacency_to_change_lower_list = hexes_adjacency_to_change_lower['pickup_location_hex_8'].unique().tolist()
hexes_adjacency_to_change_upper_list = hexes_adjacency_to_change_upper['pickup_location_hex_8'].unique().tolist()

lower_adjacency = pd.read_csv('lower_half_full.csv')
lower_adjacency_list = lower_adjacency['pickup_location_hex_8'].unique().tolist()
upper_adjacency = pd.read_csv('upper_right.csv')
upper_adjacency_list = upper_adjacency['pickup_location_hex_8'].unique().tolist()

print(hexes_adjacency_to_change_lower_list)
print(hexes_adjacency_to_change_upper_list)
print(lower_adjacency_list)
print(upper_adjacency_list)
print(len(hexes_adjacency_to_change_lower_list))
print(len(hexes_adjacency_to_change_upper_list))
print(len(lower_adjacency_list))
print(len(upper_adjacency_list))

['886189255bfffff', '88618924edfffff', '886189242bfffff', '886189242dfffff', '8861892421fffff', '8861892423fffff', '8861892405fffff', '8861892429fffff', '88618924e5fffff', '886189240dfffff', '8861892435fffff', '8861892465fffff', '8861892467fffff', '8861892553fffff', '886189246bfffff', '8861892461fffff', '88618925d9fffff', '8861892463fffff', '8861892437fffff', '88618925dbfffff', '88618924e7fffff', '886189243dfffff', '8861892433fffff', '8861892403fffff', '886189243bfffff', '8861892445fffff', '8861892439fffff', '8861892415fffff', '8861892431fffff', '8861892447fffff', '88618924adfffff', '8861892407fffff', '8861892469fffff', '886189246dfffff', '8861892401fffff']
['886189258dfffff', '8861892513fffff', '88618925ebfffff', '88618925e3fffff', '886189251bfffff', '88618925c7fffff', '8861892585fffff', '88618925c5fffff', '88618925e9fffff', '88618925e1fffff', '88618925cdfffff', '88618925c1fffff', '8861892519fffff', '8861892517fffff', '88618925edfffff', '8861892511fffff']
['886189255bfffff', '88618924

In [14]:
df_adjacency['adjacency'] = df_adjacency['adjacency'].apply(lambda x: ast.literal_eval(x))
df_adjacency['backup_adjacency'] = df_adjacency['backup_adjacency'].apply(lambda x: ast.literal_eval(x))
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
0,88619a8ccdfffff-Monday-morning_peak,Monday,morning_peak,88619a8ccdfffff,"[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...","[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...",adding_new_cities,08:00,12:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
1,88619a8ccdfffff-Wednesday-afternoon,Wednesday,afternoon,88619a8ccdfffff,"[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...","[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...",adding_new_cities,12:00,17:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
2,88619a8ccdfffff-Wednesday-afternoon,Wednesday,afternoon,88619a8ccdfffff,"[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...","[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...",adding_new_cities,12:00,17:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
3,88619a8ccdfffff-Wednesday-rest_morning,Wednesday,rest_morning,88619a8ccdfffff,"[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...","[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...",adding_new_cities,00:00,08:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
4,88619a8ccdfffff-Wednesday-rest_morning,Wednesday,rest_morning,88619a8ccdfffff,"[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...","[88619a8ce9fffff, 88619a8ccbfffff, 88619a8c17f...",adding_new_cities,00:00,08:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru


In [15]:
df_adjacency['adjacency'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_lower_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), str(lower_adjacency_list), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_lower_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), str(lower_adjacency_list), df_adjacency['backup_adjacency'])
df_adjacency['type'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_lower_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), 'custom_sourcing_bangalore_auto_hsr_lower_2023', df_adjacency['type'])

df_adjacency['adjacency'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_upper_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), str(upper_adjacency_list), df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_upper_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), str(upper_adjacency_list), df_adjacency['backup_adjacency'])
df_adjacency['type'] = np.where(((df_adjacency['source_hex'].isin(hexes_adjacency_to_change_upper_list)) & (df_adjacency['time_period'] == 'evening_peak') & 
(df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')), 'custom_sourcing_bangalore_auto_hsr_upper_right_2023', df_adjacency['type'])

In [16]:
both_upper_lower = list(set(hexes_adjacency_to_change_lower_list + hexes_adjacency_to_change_upper_list))
both_upper_lower_full = list(set(lower_adjacency_list + upper_adjacency_list))

In [17]:
df_non_batching_adjacency_lower = df_adjacency[(df_adjacency['time_period'].isin(['evening_peak'])) & (df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')
 & (~df_adjacency['source_hex'].isin(both_upper_lower_full)) & (df_adjacency['adjacency'].apply(lambda x: any(elem in both_upper_lower for elem in x)))]

df_non_batching_backup_adjacency_lower = df_adjacency[(df_adjacency['time_period'].isin(['evening_peak'])) & (df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')
 & (~df_adjacency['source_hex'].isin(both_upper_lower_full)) & (df_adjacency['backup_adjacency'].apply(lambda x: any(elem in both_upper_lower for elem in x)))]

In [18]:
for adjacencies in df_non_batching_adjacency_lower['adjacency'].values:
    for hex in hexes_adjacency_to_change_lower_list:
        if hex in adjacencies:
            adjacencies.remove(hex)

for backup_adjacencies in df_non_batching_backup_adjacency_lower['backup_adjacency'].values:
    for hex in hexes_adjacency_to_change_lower_list:
        if hex in backup_adjacencies:
            backup_adjacencies.remove(hex)

df_non_batching_adjacency_lower.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
542037,886189244dfffff-Friday-evening_peak,Friday,evening_peak,886189244dfffff,"[886189271bfffff, 8861892441fffff, 886189244df...","[8861892719fffff, 8861892635fffff, 8861892713f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
542053,886189244dfffff-Sunday-evening_peak,Sunday,evening_peak,886189244dfffff,"[886189271bfffff, 8861892441fffff, 886189244df...","[8861892719fffff, 8861892635fffff, 8861892713f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
546188,886189253bfffff-Saturday-evening_peak,Saturday,evening_peak,886189253bfffff,"[886189253bfffff, 8861892515fffff, 88618925edf...","[8861892515fffff, 8861892517fffff, 8861892537f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
548837,88618925abfffff-Thursday-evening_peak,Thursday,evening_peak,88618925abfffff,"[88618925a3fffff, 88618925bdfffff, 88618925abf...","[88618925b5fffff, 8861892583fffff, 8861892587f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
549028,886189241dfffff-Wednesday-evening_peak,Wednesday,evening_peak,886189241dfffff,"[8861892411fffff, 8861892419fffff, 886189240bf...","[8861892413fffff, 8861892409fffff, 8861892411f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore


In [19]:
df_non_batching_adjacency_lower = df_non_batching_adjacency_lower.rename(columns = {'adjacency':'new_adjacency'})
df_non_batching_backup_adjacency_lower = df_non_batching_backup_adjacency_lower.rename(columns = {'backup_adjacency':'new_backup_adjacency'})

df_non_batching_adjacency_lower.head()

,uid,dayname,time_period,source_hex,new_adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
542037,886189244dfffff-Friday-evening_peak,Friday,evening_peak,886189244dfffff,"[886189271bfffff, 8861892441fffff, 886189244df...","[8861892719fffff, 8861892635fffff, 8861892713f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
542053,886189244dfffff-Sunday-evening_peak,Sunday,evening_peak,886189244dfffff,"[886189271bfffff, 8861892441fffff, 886189244df...","[8861892719fffff, 8861892635fffff, 8861892713f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
546188,886189253bfffff-Saturday-evening_peak,Saturday,evening_peak,886189253bfffff,"[886189253bfffff, 8861892515fffff, 88618925edf...","[8861892515fffff, 8861892517fffff, 8861892537f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
548837,88618925abfffff-Thursday-evening_peak,Thursday,evening_peak,88618925abfffff,"[88618925a3fffff, 88618925bdfffff, 88618925abf...","[88618925b5fffff, 8861892583fffff, 8861892587f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
549028,886189241dfffff-Wednesday-evening_peak,Wednesday,evening_peak,886189241dfffff,"[8861892411fffff, 8861892419fffff, 886189240bf...","[8861892413fffff, 8861892409fffff, 8861892411f...",reset_to_original_auto,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore


In [20]:
df_adjacency = df_adjacency.merge(df_non_batching_adjacency_lower[['uid', 'time_period', 'service_detail_id', 'new_adjacency']], 
on = ['uid', 'time_period', 'service_detail_id'], how = 'left')
df_adjacency = df_adjacency.merge(df_non_batching_backup_adjacency_lower[['uid', 'time_period', 'service_detail_id', 'new_backup_adjacency']], 
on = ['uid', 'time_period', 'service_detail_id'], how = 'left')

df_adjacency['new_adjacency'] = df_adjacency['new_adjacency'].fillna('No')
df_adjacency['new_backup_adjacency'] = df_adjacency['new_backup_adjacency'].fillna('No')
df_adjacency['adjacency'] = np.where(df_adjacency['new_adjacency'] != 'No', df_adjacency['new_adjacency'], df_adjacency['adjacency'])
df_adjacency['backup_adjacency'] = np.where(df_adjacency['new_backup_adjacency'] != 'No', df_adjacency['new_backup_adjacency'], df_adjacency['backup_adjacency'])
df_adjacency['type'] = np.where(df_adjacency['new_adjacency'] != 'No', 'custom_sourcing_bangalore_2023_hex_removal_auto', df_adjacency['type'])
df_adjacency['type'] = np.where(df_adjacency['new_backup_adjacency'] != 'No', 'custom_sourcing_bangalore_2023_hex_removal_auto', df_adjacency['type'])
df_adjacency = df_adjacency[['uid', 'dayname', 'time_period', 'source_hex', 'adjacency', 'backup_adjacency', 'type', 'start_ts', 'end_ts', 'city_id', 'date', 
                             'propagation_type', 'propagation_eta_buckets', 'multicast_max_batch_size', 'service_detail_id', 'undesirable_hexes', 
                             'max_mmo_parallel_order', 'propagation_score_buckets', 'on_ride_booking_enabled', 'pap_enabled', 'pap_batch_window_in_ms', 
                             'batch_adjacency', 'route_preference_eligibility_threshold_score', 'city_name']]

df_adjacency['adjacency'] = df_adjacency['adjacency'].astype(str)
df_adjacency['backup_adjacency'] = df_adjacency['backup_adjacency'].astype(str)
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
0,88619a8ccdfffff-Monday-morning_peak,Monday,morning_peak,88619a8ccdfffff,"['88619a8ce9fffff', '88619a8ccbfffff', '88619a...","['88619a8ce9fffff', '88619a8ccbfffff', '88619a...",adding_new_cities,08:00,12:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
1,88619a8ccdfffff-Wednesday-afternoon,Wednesday,afternoon,88619a8ccdfffff,"['88619a8ce9fffff', '88619a8ccbfffff', '88619a...","['88619a8ce9fffff', '88619a8ccbfffff', '88619a...",adding_new_cities,12:00,17:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
2,88619a8ccdfffff-Wednesday-afternoon,Wednesday,afternoon,88619a8ccdfffff,"['88619a8ce9fffff', '88619a8ccbfffff', '88619a...","['88619a8ce9fffff', '88619a8ccbfffff', '88619a...",adding_new_cities,12:00,17:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
3,88619a8ccdfffff-Wednesday-rest_morning,Wednesday,rest_morning,88619a8ccdfffff,"['88619a8ce9fffff', '88619a8ccbfffff', '88619a...","['88619a8ce9fffff', '88619a8ccbfffff', '88619a...",adding_new_cities,00:00,08:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru
4,88619a8ccdfffff-Wednesday-rest_morning,Wednesday,rest_morning,88619a8ccdfffff,"['88619a8ce9fffff', '88619a8ccbfffff', '88619a...","['88619a8ce9fffff', '88619a8ccbfffff', '88619a...",adding_new_cities,00:00,08:00,5d440361a0fb110efd865743,20211105,unicast,"[[0,6],[6,12],[12,18]]",5,5d440cc9e63140173ee1beea,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,eluru


In [21]:
for x in df_adjacency['backup_adjacency']:
    if "\n" in x:
        print(x)

In [22]:
df_adjacency.shape

(7353465, 24)

In [23]:
df_adjacency.to_parquet('pan_india_adjacency_altered_20230509.parquet', index = False)

In [26]:
df_adjacency[(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_lower_2023')]

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
447237,8861892415fffff-Saturday-evening_peak,Saturday,evening_peak,8861892415fffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
447265,8861892415fffff-Thursday-evening_peak,Thursday,evening_peak,8861892415fffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
448309,8861892401fffff-Saturday-evening_peak,Saturday,evening_peak,8861892401fffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
448653,886189242bfffff-Friday-evening_peak,Friday,evening_peak,886189242bfffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
542820,886189246bfffff-Saturday-evening_peak,Saturday,evening_peak,886189246bfffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743910,886189242bfffff-Tuesday-evening_peak,Tuesday,evening_peak,886189242bfffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
5743966,886189242bfffff-Sunday-evening_peak,Sunday,evening_peak,886189242bfffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
5767070,8861892437fffff-Monday-evening_peak,Monday,evening_peak,8861892437fffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
5767174,88618924edfffff-Friday-evening_peak,Friday,evening_peak,88618924edfffff,"['886189255bfffff', '88618924edfffff', '886189...","['886189255bfffff', '88618924edfffff', '886189...",custom_sourcing_bangalore_auto_hsr_lower_2023,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c5

In [29]:
16*7

112

In [27]:
df_adjacency[(df_adjacency['type'] == 'custom_sourcing_bangalore_auto_hsr_upper_right_2023')]

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,city_name
448476,88618925c1fffff-Saturday-evening_peak,Saturday,evening_peak,88618925c1fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
448681,88618925c5fffff-Thursday-evening_peak,Thursday,evening_peak,88618925c5fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
448781,88618925c1fffff-Friday-evening_peak,Friday,evening_peak,88618925c1fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
542136,8861892513fffff-Wednesday-evening_peak,Wednesday,evening_peak,8861892513fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
542209,8861892513fffff-Saturday-evening_peak,Saturday,evening_peak,8861892513fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527253,8861892519fffff-Monday-evening_peak,Monday,evening_peak,8861892519fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.0,bangalore
5603528,8861892511fffff-Wednesday-evening_peak,Wednesday,evening_peak,8861892511fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
5682111,88618925e9fffff-Thursday-evening_peak,Thursday,evening_peak,88618925e9fffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,True,3000,,0.0,bangalore
5764238,88618925ebfffff-Thursday-evening_peak,Thursday,evening_peak,88618925ebfffff,"['886189258dfffff', '8861892513fffff', '886189...","['886189258dfffff', '8861892513fffff', '886189...",custom_sourcing_bangalore_auto_hsr_upper_right...,17:00,22:00,572ca7ff116b5db3057bd

In [55]:
new_adjacency = pd.read_parquet('pan_india_adjacency_altered_20230427.parquet')
new_adjacency.head()

: 

: 

In [ ]:
new_adjacency['time_period'].value_counts()

time_period
rest_morning    1421687
rest_evening    1421675
morning_peak    1421671
evening_peak    1421671
afternoon       1421621
Name: count, dtype: int64

In [ ]:
new_adjacency['dayname'].value_counts()

dayname
Friday       1015483
Tuesday      1015481
Sunday       1015481
Wednesday    1015479
Thursday     1015475
Monday       1015471
Saturday     1015455
Name: count, dtype: int64

In [ ]:
df_adjacency['type'] == 'custom_sourcing_bangalore_2023'